<a href="https://colab.research.google.com/github/trishac97/ML-Policy-PoW/blob/master/IP_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pickle
import gensim
import random
import csv
import numpy as np
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
goodIP_file_path = '/gdrive/MyDrive/IP_Classifier/DAbR-Dataset-main/good_IP_data.csv'
badIP_file_path = '/gdrive/MyDrive/IP_Classifier/DAbR-Dataset-main/bad_IP_data.csv'

In [ ]:
def list_splitter(list_to_split, ratio):
    elements = len(list_to_split)
    middle = int(elements * ratio)
    return [list_to_split[:middle], list_to_split[middle:]]

In [ ]:
with open(goodIP_file_path, encoding="ISO-8859-1", newline='') as f:
    reader = csv.reader(f)
    goodIpList = list(reader)
with open(badIP_file_path, encoding="ISO-8859-1", newline='') as f:
    reader = csv.reader(f)
    badIpList = list(reader)   
ipAttributeList = goodIpList[0] # ['ip_address', 'city.names.en', 'subdivisions.names.en', 'country.iso_code', 'registered_country.iso_code', 'traits.user_type', 'traits.asn', 'traits.isp']
goodIpList = goodIpList[1:]
badIpList = badIpList[1:]

mergedIpList = goodIpList + badIpList

In [ ]:
goodIpTrainList, goodIpTestList = list_splitter(goodIpList, 0.75)
badIpTrainList, badIpTestList = list_splitter(badIpList, 0.75)

In [ ]:
attributeDict = [{}, {}, {}, {}, {}, {}, {}]

for i in goodIpList:
  for j in range(1,8):
    if i[j] not in attributeDict[j-1].keys():
      attributeDict[j-1][i[j]] = 0


for i in badIpList:
  for j in range(1,8):
    if i[j] not in attributeDict[j-1].keys():
      attributeDict[j-1][i[j]] = 1/(len(mergedIpList))
    else:  
      attributeDict[j-1][i[j]] = attributeDict[j-1][i[j]] + 1/len(mergedIpList)

In [ ]:
attributeDict[2]['VN']

0.017199325248473177

In [ ]:
maxBadIpVector = [0, 0, 0, 0, 0, 0, 0]
indx = 0

for i in attributeDict:
  maxBadIpVector[indx] = max(i.values())
  indx += 1

In [ ]:
maxBadIpVector

[0.2798281207257081,
 0.2775827482446257,
 0.28741907540794615,
 0.28754445153633146,
 0.576639007932783,
 0.22207531685955978,
 0.09673338196407746]

In [ ]:
def getIpVector(ipAttrubute):
  ipVector = [0, 0, 0, 0, 0, 0, 0]
  indx = 0
  for i in ipAttrubute:
    try:
      ipVector[indx] = attributeDict[indx][i]
    except:
      ipVector[indx] = 0
    indx += 1
  return ipVector

In [ ]:
def getReputationScore(ipAttrubute):
  ed = np.linalg.norm(np.array(getIpVector(ipAttrubute)) - np.array([0, 0, 0, 0, 0, 0, 0]))
  edMax = np.linalg.norm(np.array(maxBadIpVector) - np.array([0, 0, 0, 0, 0, 0, 0]))

  dabrScore = (1 - (ed/edMax)) * 10

  return dabrScore

In [ ]:
ipClassifyResult = []

for i in goodIpTestList:
  score = getReputationScore(i[1:])
  if score > 6:
    ipClassifyResult.append(1)
  else:
    ipClassifyResult.append(0)

for i in badIpTestList:
  score = getReputationScore(i[1:])
  if score > 6:
    ipClassifyResult.append(0)
  else:
    ipClassifyResult.append(1)

sum(ipClassifyResult)/len(ipClassifyResult)

0.8036927285160702

**Below is old code... No need to execute...**

In [ ]:
goodIpVector = []
badIpVector = []


for i in goodIpTrainList:
  temp = [0, 0, 0, 0, 0, 0, 0]
  for j in range(1,8):
    temp[j-1] = attributeDict[j-1][i[j]]
    goodIpVector.append(temp)

for i in badIpTrainList:
  temp = [0, 0, 0, 0, 0, 0, 0]
  for j in range(1,8):
    temp[j-1] = attributeDict[j-1][i[j]]
    badIpVector.append(temp)

In [ ]:
goodIpVector[0]

[0, 0, 0, 0, 0, 0, 0]

In [ ]:
goodIpVector = np.array(goodIpVector)
goodIpMean = goodIpVector.sum(axis=0)
goodIpMean = np.true_divide(goodIpMean, len(goodIpVector))

badIpVector = np.array(badIpVector)
badIpMean = badIpVector.sum(axis=0)
badIpMean = np.true_divide(badIpMean, len(badIpMean))

In [ ]:
print(goodIpMean)

[104.42384665  28.94561404   9.13996101  11.48135153   1.80838207
 539.81234568 559.391423  ]


In [ ]:
print(badIpMean)

[3.81064730e+07 7.11621000e+06 9.21043000e+05 9.01276000e+05
 4.54950000e+04 8.67066620e+07 1.04654919e+08]


In [ ]:
goodIpTestVector = []
badIpTestVector = []


for i in goodIpTestList:
  temp = [0, 0, 0, 0, 0, 0, 0]
  for j in range(1,8):
    temp[j-1] = attributeDict[j-1][i[j]]
    goodIpTestVector.append(temp)

for i in badIpTestList:
  temp = [0, 0, 0, 0, 0, 0, 0]
  for j in range(1,8):
    temp[j-1] = attributeDict[j-1][i[j]]
    badIpTestVector.append(temp)

In [ ]:
ipClassifyResult = []

for i in goodIpTestVector:
  if (np.linalg.norm(i-goodIpMean) / (np.linalg.norm(i-goodIpMean) + np.linalg.norm(i-badIpMean))) < 0.99:
    ipClassifyResult.append(1)
  else:
    ipClassifyResult.append(0)

for i in badIpTestVector:
  if (np.linalg.norm(i-goodIpMean) / (np.linalg.norm(i-goodIpMean) + np.linalg.norm(i-badIpMean))) >= 0.99:
    ipClassifyResult.append(1)
  else:
    ipClassifyResult.append(0)

In [ ]:
sum(ipClassifyResult)/len(ipClassifyResult)

0.194985754985755

In [ ]:
ipClassifyResult

In [ ]:
np.linalg.norm(badIpTestVector[100]-goodIpMean)

2641.9695394420874

In [ ]:
np.linalg.norm(badIpTestVector[100]-badIpMean)

141332078.53580436